# In this script im trying to calculate the volume of the single RBC by considering the pixel to metric ratio provided by the microscope setup, which was determined to 75.5 nm per pixel

### Try it on single image

### Import dependencies

In [1]:
import os
import cv2

### Read every single frame from the tif-file and store it in a list 

In [2]:
from PIL import Image
import numpy as np
# Open the TIFF file
tiff_file = Image.open("Results/cell_frame004141_x0100_y0956_red.tif")


frames= []

# Iterate over each frame in the TIFF file
for i in range(tiff_file.n_frames):
    # Go to the current frame
    tiff_file.seek(i)

    # Convert the frame to the unsigned 8-bit integer format
    frame = tiff_file.copy()  # Convert to grayscale if needed
    frame = np.array(frame)
    frame = (frame * 255).astype(np.uint8)

    # Append the frame to the list
    frames.append(frame)
    


### find optimal threshold for volume calculation / read in "BTHE_3D_visualization.py"

In [3]:
import numpy as np
import tifffile

def threshold_volume(input_file, output_file, threshold):
    # Load the TIF volume
    volume = tifffile.imread(input_file)
    volume = volume *255

    # Apply the thresholding
    binary_volume = np.where(volume < threshold, 0, 255).astype(np.uint8)

    # Save the binary volume as a TIF file
    tifffile.imwrite(output_file, binary_volume)

# Example file
input_file = 'Results/cell_frame004144_x0181_y0104_red.tif'
output_file = 'Created_Images/output.tif'
threshold_value = 175 #optimtal threshold value


threshold_volume(input_file, output_file, threshold_value)

### Count the pixels in each frame which are above a certain threshold intensity value

In [4]:
threshold = 175

count = 0
for i in range(len(frames)):
    sum = np.sum(frames[i] > threshold)
    count = count + sum

count #amount of pixels above threshold intensity



42103

### Multiply the counted pixels with the ratio to get the real-world dimensions

In [5]:
metric_to_pixel = (75.5/1000) #75.5nm/px ; /1000 => micrometer/px; 
scaling_factor = 0.633 #from the jupyter notebook "BTHE_Altering_Images.ipynb" 
metric_to_pixel = (metric_to_pixel / scaling_factor )  ** 3 #new metric to pixel value because the input image was resized (without loosing aspect ratio); ^3 to get cubic-micrometer

volume = count * metric_to_pixel # cubic micrometer
print(int(volume))


71


## Apply to all tif-files in the predictions folder "results" and store it in a new excel

### get scaling_factor from every image and store it in a list 


In [6]:
import os
import cv2
from PIL import Image
import numpy as np
desired_width, desired_height = 64, 64
desired_aspect_ratio = desired_width / desired_height



# Set the path to the folder containing the images
folder_path = "Images_for_prediction"  

scale_factors = []


# Loop over all the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Load the original image
        file_path = os.path.join(folder_path, filename)
        original_image = Image.open(file_path)

        current_width, current_height = original_image.size
        current_aspect_ratio = current_width / current_height

        if current_aspect_ratio > desired_aspect_ratio:
            # The original image is wider than the desired aspect ratio, so the width will be the limiting factor.
            scale_factor = desired_width / current_width
            scale_factors.append(scale_factor)
        else:
            # The original image is taller than the desired aspect ratio, so the height will be the limiting factor.
            scale_factor = desired_height / current_height
            scale_factors.append(scale_factor)


print(len(scale_factors))
print(scale_factors[:5])

11668
[0.6336633663366337, 0.6037735849056604, 0.5614035087719298, 0.5245901639344263, 0.5517241379310345]


### Calculate the volume based on scale_factors and store volumes in new excel file in the folder "Features"

In [7]:
from PIL import Image
import numpy as np
import openpyxl
threshold = 175 #manually defined as good  by experimenting with different values



folder_path = "Results"
out_path = "Data_Analysis"

# Create a new workbook and get the active sheet
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.column_dimensions["A"].width = 100


sheet["A1"] = "filename"
sheet["B1"] = "image_scale_factor"
sheet["C1"] = "count"
sheet["D1"] = "metric_to_pixel"
sheet["E1"] = "volume"

# Loop over all the prediction in the "Results" folder
for index, filename in enumerate(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    # Open the TIFF file
    tiff_file = Image.open(file_path)

    frames= []

    # Iterate over each frame in the TIFF file
    for i in range(tiff_file.n_frames):


        
        # Go to the current frame
        tiff_file.seek(i)

        # Convert the frame to the unsigned 8-bit integer format
        frame = tiff_file.copy()  # Convert to grayscale if needed
        frame = np.array(frame)
        frame = (frame * 255).astype(np.uint8)

        # Append the frame to the list
        frames.append(frame)


    count = 0
    for i in range(len(frames)):
        sum = np.sum(frames[i] > threshold)
        count = count + sum
    
    metric_to_pixel = (75.5/1000) #75.5nm/px ; /1000 => micrometer/px; 
    scaling_factor = scale_factors[index] #from the jupyter notebook "BTHE_Altering_Images.ipynb" 
    metric_to_pixel = (metric_to_pixel / scaling_factor )  ** 3 #new metric to pixel value because the input image was resized (without loosing aspect ratio); ^3 to get cubic-micrometer

    volume = count * metric_to_pixel # cubic micrometer

    # Write data rows in excel worksheet
    row = [filename, scaling_factor,count, metric_to_pixel, volume]
    sheet.append(row)

output_file = os.path.join(out_path, 'Volume_Calculation_manualThresh.xlsx')
workbook.save(output_file)



    
        